# Datasets e DataLoaders no Pytorch

Demonstração do funcionamento dos Datasets e Dataoaders no Pytorch

## Datasets

No Pytorch, datasets são objetos utilizados para armazenar, indexar e e retornar elementos de um conjunto de dados.

Usualmente eles devem ter funções para retornar o elemento de um índice e deve ser possível saber o tamanho do dataset.

### Criando um Dataset com torch.utils.data.TensorDataset

Se os dados e as classes forem disponibilizados em arrays do numpy ou tensores do próprio PyTorch é possível criar um dataset utilizando [torch.utils.data.TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset):

In [70]:
import numpy as np
import torch
from torch.utils.data import TensorDataset

# suposta entrada  
x_data = np.random.rand(20, 3)                 # 20 vetores de 3 elementos criados aleatoriamente
target = np.random.randint(0, 3, size=20)    # 20 classes 0, 1, ou 2 criadas aleatoriamente

print('dimensões de x_data:', x_data.shape)
print('dimensões de target:', target.shape)

dimensões de x_data: (20, 3)
dimensões de target: (20,)


In [71]:
# passo 1: transformar em tensores torch
x_data = torch.from_numpy(x_data)
target = torch.from_numpy(target)

# passo 2: usar TensorDataset para criar o dataset com os tensores
dataset = TensorDataset(x_data, target)

Cada elemento do dataset retornará o dado e a classe do índice pedido em uma tupla

In [72]:
i = 10
x, y = dataset[i]

print('dado:', x)
print('classe:', y)

dado: 
 0.9509
 0.4563
 0.6243
[torch.DoubleTensor of size 3]

classe: 1


### Criando um Dataset com torch.utils.data.Dataset

#### Estrutura básica do Dataset

Vamos criar um dataset de exemplo utilizando como classe base [torch.utils.data.Dataset](http://pytorch.org/docs/master/data.html#torch.utils.data.Dataset).

Para criar uma subclasse de `Dataset` é preciso definir as funções `__len__`, para retornar o tamanho do dataset, e `__getitem__`, para retornar um elemento de um índice dado.

A função `__len__` é chamada quando usamos o método `len(dataset)` do Python e `__getitem__` é chamada quando fazemos a indexação `dataset[i]`.

In [12]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __len__(self):
        pass
    
    def __getitem__(self):
        pass

Acima temos a estrutura básica de uma classe do Dataset, mas precisamos de dados, por isso vamos criar dados aleatórios para testar o seu funcionamento.

In [38]:
### from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
       
        # gera 20 vetores de 3 elementos
        self.data = torch.rand(20, 3)
    
    def __len__(self):
        pass
    
    def __getitem__(self):
        pass

No código acima utilizamos o método [torch.rand](http://pytorch.org/docs/master/torch.html#torch.rand) para gerar 20 vetores de 3 elementos (ou uma matrix 20x3) no construtor da classe, que serão nossos dados. 

Podemos, então, definir  `__len__` e `__getitem__`:

In [34]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
        
        # gera 20 vetores de 3 elementos
        self.data = torch.rand(20, 3)
    
    def __len__(self):
        # retorna o tamanho da primeira dimensão
        return self.data.size(0)
    
    def __getitem__(self, i):
        # retornar a i-ésima matriz 3x3 de data
        return self.data[i, :]

Nossa classe está pronta, podemos criar um objeto dela.

In [35]:
# Criando um objeto da nossa classe
dataset = MyDataset()

Nosso método `__len__` deve retornar o tamanho da primeira dimensão dos dados (20), que é o tamanho de vetores de 3 elementos no dataset.

Podemos verificar se o método está correto chamando o `len()` do Python:

In [39]:
print(len(dataset))

20


Nosso método `__getitem__()` deve retornar o elemento na i-ésima posição no dataset, no nosso caso `data[i]`:

In [41]:
i = 1
print(dataset[i])
# é equivalente à:
print(dataset.data[i])


 0.0516
 0.4421
 0.8458
[torch.FloatTensor of size 3]


 0.0516
 0.4421
 0.8458
[torch.FloatTensor of size 3]



#### Incluindo a classe dos dados

Um dataset também deve contém a classe a qual o dado pertence, a seguir incluiremos classes ao exemplo anterior.

In [47]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
        
        # gera 20 vetores de 3 elementos
        self.data = torch.rand(20, 3)
        # gera as 20 elementos contendo classes 0, 1 ou 2
        self.classes = (torch.rand(20)*3).type(torch.LongTensor)
    
    def __len__(self):
        # retorna o tamanho da primeira dimensão
        return self.data.size(0)
    
    def __getitem__(self, i):
        # retornar a i-ésima matriz 3x3 de data
        return (self.data[i, :], self.classes[i])

Agora nosso método `__getitem__` retorna uma tupla (dado, classe).

In [53]:
dataset = MyDataset()

data, label = dataset[1]

print('dado:', data)
print('classe:', label)

dado: 
 0.9795
 0.7468
 0.1999
[torch.FloatTensor of size 3]

classe: 0


#### Conclusão

O caso apresentado neste tópico é muito simples, apresentando os conceitos básicos da estrutura que o Pytorch utiliza para representar datasets. A classe torch.utils.data.Dataset é usada para casos complexos em que é preciso, por exemplo, carregar o dataset de arquivos e quando o gerenciamento dos dados não é tão trivial.

## DataLoaders

Um DataLoader ([torch.utils.data.DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader)) combina um Dataset e um Sampler (divide os dados em batches). Ele torna possível iterar sobre os dados utilizando vários processos (multi-process iterator) tornando o treinameto mais simples e rápido.

### Criando um DataLoader

Inicialmente vamos definir um dataset para ser a fonte de dados do nosso DataLoader.

In [78]:
import numpy as np
import torch
from torch.utils.data import TensorDataset

# dados aleatorios  
x_data = np.random.rand(20, 3, 3)            # 20 matrizes de 3x3 criadas aleatoriamente
target = np.random.randint(0, 3, size=20)    # 20 classes 0, 1, ou 2 criadas aleatoriamente

print('dimensões de x_data:', x_data.shape)
print('dimensões de target:', target.shape)

dimensões de x_data: (20, 3, 3)
dimensões de target: (20,)


In [79]:
# transforma em tensores torch
x_data = torch.from_numpy(x_data)
target = torch.from_numpy(target)

# usar TensorDataset para criar o dataset com os tensores
dataset = TensorDataset(x_data, target)

print('tamanho do dataset: ', len(dataset))

tamanho do dataset:  20


Agora podemos criar o DataLoader com o dataset

In [171]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, 
                         batch_size=5,           # tamanho do batch de dados
                         shuffle=False,          # se for True, embaralha os dados no inicio de cada iteração
                         num_workers=2)          # número de processos criados para pegar os dados

### Iterando sobre o DataLoader

Podemos iterar sobre o DataLoader utilizando um `for`

In [149]:
batch_n = 0

for data in data_loader:
    print('batch ', batch_n)
    batch_n += 1
    
print('tamanho do DataLoader', len(data_loader))

batch  0
batch  1
batch  2
batch  3
tamanho do DataLoader 4


O tamanho do DataLoader é 4, pois temos batches de tamanho 5 e 20 dados no dataloader.

Agora podemos utilizar os dados do DataLoader:

In [150]:
batch_n = 0

for data in data_loader:
    # separa a tupla em dados e classes
    data_batch, targets_batch = data
    
    print('dimensão do batch de dados {}:   {}'.format(batch_n, data_batch.size()))
    print('dimensão do batch de classes {}: {}\n'.format(batch_n, targets_batch.size()))
    batch_n += 1


dimensão do batch de dados 0:   torch.Size([5, 3, 3])
dimensão do batch de classes 0: torch.Size([5])

dimensão do batch de dados 1:   torch.Size([5, 3, 3])
dimensão do batch de classes 1: torch.Size([5])

dimensão do batch de dados 2:   torch.Size([5, 3, 3])
dimensão do batch de classes 2: torch.Size([5])

dimensão do batch de dados 3:   torch.Size([5, 3, 3])
dimensão do batch de classes 3: torch.Size([5])



É possível ver que os batches tem mesmo 5 dados (5 matrizes 3x3 e 5 classes)

### Exercício

Defina seu próprio DataLoader na segunda célula abaixo. Faça com que cada batch tenha 10 matrizes 3x3. Utilize o dataset já criado anteriormente.

Em seguida complete o loop que itera sobre o seu DataLoader para que seja possível plotar as matrizes de cada batch. Utilize a função plot_batch para plotar as matrizes de um batch inteiro.

In [176]:
# Código usado no exercício para plotar as imagens, NÃO É NECESSÁRIO ALTERAR
import matplotlib.pyplot as plt
import math 

def plot_batch(batch, batch_n):
    fig = plt.figure(figsize=(5,2))
    for j, mat in enumerate(batch):
        plt.subplot(1, batch.size(0), j+1)
        plt.imshow(mat.numpy(), cmap='gray')
        plt.axis('off')
    plt.suptitle('batch {} - {} elementos'.format(batch_n, batch.size(0)))
    plt.show()

In [177]:
# -- Defina aqui seu DataLoader com batches de 10 elementos cada ---
data_loader = None
    
# -- Defina aqui o loop sobre todo o DataLoader dando plot em cada batch com os dados
for data in []:
    # separe o dado das classes
    
    plot_batch([], batch_n)

- A ordem dos elementos se altera entre execuções da célula acima?
- Tente colocar o parametro `shuffle` do DataLoader como `True` e obeseve o resultado do exercício, rodando várias vezes.